### Sources:

- https://towardsdatascience.com/genetic-algorithm-implementation-in-python-5ab67bb124a6

In [34]:
import numpy as np
import pandas as pd
import svvamp as sv
from whalrus.profile.Profile import Profile
from whalrus.rule.RuleCondorcet import RuleCondorcet
from whalrus.rule.RuleScorePositional import RuleScorePositional

### Election code

In [159]:
def label_profile(profile, candidates):
    """ Convert profile to a list of {candidate: rank} per voter
        Then return a list of labeled candidates in order of rank """

    ordered_prof = []

    for ballot in profile:
        ordered_ballot = {}
        for candidate, rank in zip(candidates, ballot):
            ordered_ballot[candidate] = rank

        ordered_prof.append(ordered_ballot)

    sorted_dicts = [dict(sorted(profile.items(), key=lambda kv: kv[1])) for profile in ordered_prof]
    sorted_dict_keys = [list(d.keys()) for d in sorted_dicts]

    return sorted_dicts, sorted_dict_keys


def profile_as_matrix(profile):
    """ Create a matrix representation of the profile,
    where matrix[c][r] represent the frequency of candidate c in rank r """

    matrix_rank = [[0] * len(profile[0]) for _ in profile[0]]

    for ballot in profile:
        for j, rank in enumerate(ballot):
            matrix_rank[j][rank] += 1

    return np.array(matrix_rank)


def profile_as_dataframe(matrix_rank, candidates):
    """ Creates a dataframe representation of the profile from the matrix representation """
    data_dict = dict()

    # Create dictionary for dataframe
    data_dict["Candidates"] = candidates
    for i in range(len(matrix_rank)):
        data_dict[f"{i}th rank"] = matrix_rank[:, i]

    df = pd.DataFrame(data_dict)

    return df


def create_profile_from_distribution(n_voters, candidates, pop_type="spheroid"):
    """ Return a Profile object given a number of voters, a list of candidates and a population type """

    # TODO: INSERT IF STATEMENTS FOR MULTIPLE POPULATION TYPES if necessary
    # Create a population object
    pop = sv.PopulationSpheroid(V=n_voters, C=len(candidates))
    pop.labels_candidates = candidates

    # Create a dataframe representation of the profile so we can print it out
    matrix_rank = profile_as_matrix(pop.preferences_rk)
    df_rank = profile_as_dataframe(matrix_rank, pop.labels_candidates)

    # Create a labeled version of the ranking from the population
    profile_dicts, labeled_ranks = label_profile(pop.preferences_rk, pop.labels_candidates)

    # Create a profile object - automatically creates ordered ballots
    return Profile(labeled_ranks), df_rank, matrix_rank

In [160]:
def election(profile, weights=None):
    if weights is None:
        raise Exception("Must insert weights!")

    rule = RuleScorePositional(profile, points_scheme=weights)

    results = dict()
    results["Gross scores"] = rule.gross_scores_
    results["Average scores"] = rule.scores_
    results["Average scores as floats"] = rule.scores_as_floats_
    results["Winner(s)"] = rule.cowinners_

    return results

### Constraints

In [161]:
def check_condorcet(profile, results):
    """ Check if a profile contains a condorcet winner and if the voting rule gave it as a winner
        Returns:
            1  if the Condorcet winner was elected
            -1 if there is a Condorcet winner but was not elected
            0  if there isn't a Condorcet winner """
    
    condorcet = RuleCondorcet(profile)
    
    if len(condorcet.cowinners_) == 1 and len(results["Winner(s)"]) == 1:
        if next(iter(results["Winner(s)"])) == next(iter(condorcet.cowinners_)):
            return 1
        else:
            return -1
    return 0

def check_majority(profile_df, results):
    """ Check if the voting rule outputs a candidate that is preferred (ranked first) by the majority of the population
        We will look into the first column of the profile matrix where matrix[i][0] represents how many people 
        voted for candidate i as first in their ranking, and compare it with the winner
        
        Returns: 
            1  if majority criterion was satisfied 
            -1 if it wasn't
            0  if there isn't a clear majority winner """
    
    pop_number = sum(profile_df.iloc[0, 1:].astype(int))
    
    first_candidates = profile_df.iloc[:, 1]
    
    for i, candidate in enumerate(first_candidates):
        if candidate >= 0.5*pop_number:
            if profile_df.iloc[i, 0] == next(iter(results["Winner(s)"])):
                return 1
            else:
                return -1
    return 0
    

### Genetic Algorithm

In [162]:
def initialize(low, high, num_solutions, num_weights):
    '''
    A population will be a set of weight vectors 
    - num_solutions is the number of weight vectors 
    - num_weights is the number of weights
    - low and high are the bounds for the weight values
    '''
    pop_size = (num_solutions, num_weights)
    new_pop = np.random.uniform(low=low, high=high, size=pop_size)
    
    return new_pop

In [180]:
def crossover(parents, size, point="single"):
    offspring = np.empty(size)
    
    # TODO: Afterwards add more options
    if point=="single":
        # The point in the chromosome after which crossover occurs (if single_point, usually half)
        crossover_point = np.uint8(size[1]/2)
    
    # For every new weight vector
    for i in range(size[0]): 
        
        # Pick the ith and ith+1 parents
        parent1_idx = i % parents.shape[0]
        parent2_idx = (i+1) % parents.shape[0]
        
        # For ith offspring, first half of genes from first parent, second half from second
        offspring[i, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[i, crossover_point:] = parents[parent2_idx, crossover_point:]
        
    return offspring

In [177]:
def mutation(offspring_crossover, mutation_idx, low, high):
    ''' Change one gene at mutation_idx from a given offspring randomly 
        Value for mutation is in the range (low, high, 1) '''
    
    for i in range(offspring_crossover.shape[0]):
        random_val = np.random.uniform(low, high, 1)
        offspring_crossover[i : mutation_idx] += random_val
    
    return offspring_crossover

In [165]:
''' Fairness Constraints:

    CC = Condorcet complicity 
    MR = Majority rule
    MT = Monotonicity
    IIA = Independence of Irrelevant Alternatives 
    
    Non-manipulability Constraints:
    
    IM = Individual manipulation
    CM = Coalition manipulation
    TM = Trivial manipulation
    UM = Unison manipulation
    
    Other constraints:
    
    PC = Population Consistency
    CS = Composition Consistency
    
'''

def fitness(pop, fairness=["CC", "MR"], non_manipulablity=["IM"], others=[]):
    
    ''' Evaluate the fitness of the weight population by running an election 
        and a test of the results based on the input fairness and non-manipulability constraints '''
    
    # Profile for election
    profile, profile_df, profile_matrix = create_profile_from_distribution(500, ["Adam", "Bert", "Chad", "Derek", "Evan"])
    
    fitness_pop = []
    
    for weights in pop:
        results = election(profile, weights)
    #     for result, value in results.items():
    #         print(f"{result}: {value}")


        # Score = condorcet compliance + majority rule
        condorcet_score = check_condorcet(profile, results)
        majority_score = check_majority(profile_df, results)
        
        fitness_pop.append(condorcet_score + majority_score)

    #     print("Is Condorcet compliant?:", check_condorcet(profile, results))
    #     print("Satisfies majority criterion?", check_majority(profile_df, results))
    
    # SHOULD RETURN NUMPY ARRAY OF SCORES PER ALL THE WEIGHTS IN THE POPULATION
    return np.array(fitness_pop)
    

In [166]:
def select_mating_pool(pop, pop_fitness, n_parents):
    
    ''' Select the best n_parents among the population according to their pop_fitness '''
    parents = np.empty((n_parents, pop.shape[1])) # (n_parents, n_weights)
    
    # For every possible parent that we could to take
    for parent_i in range(n_parents):
        
        # Get the ith best fitness parent index 
        ##### TODO: What happens if multiple people have that index????
        max_fitness_idx = np.where(pop_fitness == np.max(pop_fitness))[0][0]
        
        # Get the parent with that index from pop and set it in the parents array
        parents[parent_i, :] = pop[max_fitness_idx, :]
        
        # Make sure that parent is not picked again 
        #### ALSO TODO: What happens if there are repetitions?
        pop_fitness[max_fitness_idx] = -999999999
        
    return parents

In [183]:
def genetic_main():
    low, high = -10, 10
    n_sols, n_weights = 10, 3
    n_generations = 10
    n_parents = 4
    
    pop = initialize(low, high, n_sols, n_weights)
    
    for generation in range(n_generations):
        
        # Measure the fitness of each chromosome in the population
        pop_fitness = fitness(pop)
        
        print("Generation:", generation)
        print("Weights:", pop)
        print("Fitness:", pop_fitness)
        print("-------------------------")
        
        # Select the best parents in the population for mating
        parents = select_mating_pool(pop, pop_fitness, n_parents)
        
        # Create next generation with crossover
        offspring_crossover = crossover(parents, size=(n_sols - parents.shape[0], n_weights))
        
        # Add mutations
        offspring_mutation = mutation(offspring_crossover, 3, -25, 25)
        
        # Create next population with new parents and offspring
        pop[0:parents.shape[0], :] = parents  # Set parents
        pop[parents.shape[0]:, :] = offspring_mutation  # Set offspring
        
        
        

In [184]:
genetic_main()

Generation: 0
Weights: [[ 0.46648646 -2.46887781 -4.77027006]
 [ 1.08546833  7.48785799  8.19645612]
 [-3.21114239  8.07372571 -7.29793992]
 [-5.50595003  6.95577954 -4.895909  ]
 [ 9.90576355 -0.62996296  0.22169731]
 [-7.86132021  9.81448041  2.44624496]
 [ 8.15065555 -6.0388951   5.05391999]
 [ 7.65032054 -0.39636971 -8.99273863]
 [ 2.89918613  8.21316632  8.39515899]
 [ 9.85307268 -6.77783996  8.31892236]]
Fitness: [0 0 0 0 0 0 0 0 0 0]
-------------------------
Generation: 1
Weights: [[ 0.46648646 -2.46887781 -4.77027006]
 [ 1.08546833  7.48785799  8.19645612]
 [-3.21114239  8.07372571 -7.29793992]
 [-5.50595003  6.95577954 -4.895909  ]
 [20.38404114 27.40541267 28.1140108 ]
 [32.52344424 39.51170162 24.14003599]
 [45.3578016  55.52472353 43.67303499]
 [-5.50595003 -2.46887781 -4.77027006]
 [ 0.46648646  7.48785799  8.19645612]
 [ 1.08546833  8.07372571 -7.29793992]]
Fitness: [-1 -1  1 -1 -1  1 -1 -1 -1  1]
-------------------------
Generation: 2
Weights: [[ -3.21114239   8.073725